In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 17.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from IPython.display import display, HTML
from datasets import load_dataset

In [ ]:
prompts = []

# Load Prompts

This is done after mounting drive, and checking Shareddrives > CS_263_Shared > ARC_Dataset

## Science Questions

In [ ]:
ARC_dataset_path = "/content/drive/Shareddrives/CS_263_Shared/ARC_Dataset/ARC-Challenge-Train.csv"

In [ ]:
df = pd.read_csv(ARC_dataset_path)
grade_9_science_rows = df[df['schoolGrade'] == 9]  # Just to get the hardest questions
prompts += grade_9_science_rows['question'].tolist()[:25]

## English Questions

These are just english quotes, but the idea is to use them for text summarization. Each sample has a quote and a label, where the label is either 0, 1, or 2 and corresponds to the reading level of the quote. (Elementary, intermediate, advanced)

https://huggingface.co/datasets/onestop_english

In [ ]:
english_dataset = load_dataset("onestop_english").shuffle(seed=0)

Generating train split:   0%|          | 0/567 [00:00<?, ? examples/s]

Dataset onestop_english downloaded and prepared to /root/.cache/huggingface/datasets/onestop_english/default/1.1.0/6b19eec5680862ad1cf1990e98b06a98d1fa4c85f3585dc4dfab93f52b89d9cf. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
advanced_quotes = english_dataset.filter(lambda sample: sample['label'] == 2)['train']['text']

Filter:   0%|          | 0/567 [00:00<?, ? examples/s]

In [ ]:
summarization_prompts = []
instruction_string = "Please summarize this concisely: "

for quote in advanced_quotes[:25]:
  summarization_prompts.append(instruction_string + quote)

prompts += summarization_prompts

## Coding Prompts

A mix of asking the model to write functions and debug in various languages.
https://huggingface.co/datasets/HuggingFaceH4/code_evaluation_prompts

In [ ]:
coding_dataset = load_dataset("HuggingFaceH4/code_evaluation_prompts")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/115 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/HuggingFaceH4___parquet/HuggingFaceH4--code_evaluation_prompts-f8eddf6c77719084/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
coding_prompts = coding_dataset.shuffle(seed=0)['train']['prompt'][:25]
prompts += coding_prompts

## History Prompts

Multiple choice questions from https://huggingface.co/datasets/joey234/mmlu-high_school_world_history-original-neg-prepend

In [ ]:
world_history_dataset = load_dataset("joey234/mmlu-high_school_world_history-original-neg-prepend").shuffle(seed=0)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/205 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/joey234___parquet/joey234--mmlu-high_school_world_history-original-neg-prepend-7f74e8cdc1d38c43/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
history_prompts = []

# Choose 25 prompts
for i in range(25):
  question = world_history_dataset['test']['question'][i]
  choices = world_history_dataset['test']['choices'][i]
  combined = question + " The choices are: " + ", ".join(choices)
  history_prompts.append(combined)

prompts += history_prompts

In [ ]:
assert len(prompts) == 100, f"Improper number of prompts, should be 100 but got {len(prompts)}"

# Generate Answers
This cell will run a loop that will ask you to copy a prompt. Run the prompt through an LLM and copy paste the result here. If a box for input doesn't appear, stop the cell execution and re-run it.

In [ ]:
answers = []
for prompt in prompts:
  print(f"The prompt is:\n{prompt}")
  display(HTML(f'<button onclick="navigator.clipboard.writeText(\'{prompt}\')">Copy prompt</button>'))
  answer = input("Please enter the model's answer to the prompt:")
  answers.append(answer)
  print()

The prompt is:
A fold observed in layers of sedimentary rock most likely resulted from the (A) cooling of flowing magma. (B) converging of crustal plates. (C) deposition of river sediments. (D) solution of carbonate minerals.


Please enter the model's answer to the prompt:The correct answer is (C) deposition of river sediments.  A fold observed in layers of sedimentary rock is typically formed by tectonic forces acting on the Earth's crust over long periods of time. When layers of sedimentary rock are subjected to compressional forces, they can be bent or folded, resulting in structures like anticlines (upward folds) and synclines (downward folds).  Option (A), the cooling of flowing magma, does not directly contribute to the formation of sedimentary rock folds. Magma cooling and solidifying typically form igneous rocks, not sedimentary rocks.  Option (B), the converging of crustal plates, can cause intense compressional forces that lead to the folding of rocks, including sedimentary rocks. This process is associated with mountain building and can result in the formation of large-scale folds. However, the direct cause of the folds in sedimentary rocks is the deformation resulting from plate convergence.  Opt

Please enter the model's answer to the prompt:The signal from the brain to a muscle in the arm is transmitted by motor neurons. Motor neurons are specialized nerve cells that carry signals from the central nervous system (including the brain) to the muscles, enabling voluntary movement. These neurons transmit electrical impulses, known as action potentials, from the brain or spinal cord to the muscle fibers, causing the muscles to contract and produce movement. Sensory neurons, on the other hand, are responsible for transmitting signals from sensory receptors in the body to the central nervous system, conveying information about touch, temperature, pain, and other sensations. Interneurons are neurons that facilitate communication between sensory neurons and motor neurons within the central nervous system, but they do not directly transmit signals from the brain to muscles. Mechanoreceptor neurons are a type of sensory neuron that detect mechanical stimuli, such as touch and pressure, b

Please enter the model's answer to the prompt:The characteristic of DNA that results in cell differentiation in developing embryos is (C) which genes are active. During embryonic development, all cells in the embryo possess the same DNA, but different cell types arise due to variations in gene expression. Gene expression refers to the process by which information encoded in DNA is utilized to produce functional molecules, such as proteins. Not all genes are active, or "turned on," in every cell. Instead, specific sets of genes are selectively activated or deactivated in different cell types, leading to the development of specialized cells with distinct functions.  The regulation of gene expression is tightly controlled and influenced by various factors, including signals from neighboring cells, environmental cues, and specific transcription factors. By selectively activating certain genes and repressing others, cells in developing embryos acquire unique characteristics and differentiat

Please enter the model's answer to the prompt:If more adults than children are diagnosed with lactose intolerance, it most likely suggests that (A) the production of lactase decreases over time. Lactose intolerance is commonly seen in adulthood, particularly in certain populations. The ability to produce lactase, the enzyme responsible for breaking down lactose, tends to decrease as individuals age. This decline in lactase production is a normal phenomenon known as lactase non-persistence.  During infancy and early childhood, the majority of individuals can digest lactose without any issues because they produce sufficient amounts of lactase. However, as they grow older, some individuals experience a decrease in lactase production, resulting in difficulty digesting lactose. This decrease in lactase activity can lead to symptoms such as bloating, gas, abdominal pain, and diarrhea when lactose is consumed.  It's important to note that lactose intolerance is not an allergic reaction (optio

...

Please enter the model's answer to the prompt:The speech as a whole is best understood in the context of Stalin's implementation of five-year plans for the Soviet economy. In the excerpt, Stalin emphasizes the need for rapid industrial and technological advancement to overcome the backwardness of Russia and avoid being beaten by other nations. This aligns with Stalin's push for rapid industrialization and modernization through the implementation of five-year plans, which aimed to transform the Soviet Union into a leading industrial power. The speech reflects Stalin's drive to motivate Soviet industry and society to achieve these ambitious economic goals.

The prompt is:
This question refers to the following information.
"The struggle against neo-colonialism is not aimed at excluding the capital of the developed world from operating in less developed countries. It is aimed at preventing the financial power of the developed countries being used in such a way as to impoverish the less dev

Please enter the model's answer to the prompt:The national plan advanced by Nkrumah in the second paragraph is most inspired by Socialism. Nkrumah's emphasis on cooperation with all states, including capitalist and socialist ones, and the need for foreign investment to be aligned with a national plan drawn up by the government indicates a socialist approach to economic planning and development. In socialism, the government plays a significant role in guiding the economy and ensuring that investments align with the interests of the nation. Nkrumah's emphasis on the issue of power and the need for a non-aligned state to be the master of its own destiny aligns with socialist ideals of economic self-determination and challenging the influence of foreign capital.

The prompt is:
This question refers to the following information.
Is it not unity alone that can weld us into an effective force, capable of creating our own progress and making our valuable contribution to world peace? Which inde

Please enter the model's answer to the prompt:The passage does not directly mention the factors preventing the fulfillment of the political dreams described. However, based on the given options, the factor that aligns most closely with the information provided is "A lack of common ethnicity, language, or religion." The passage emphasizes the need for unity and common purpose among independent African states, suggesting that a lack of shared ethnic, linguistic, or religious identity may have hindered the realization of their political aspirations.

The prompt is:
This question refers to the following information.
The following poem refers to an incident in Sharpeville, South Africa, in 1960. To protest the requirement that they carry papers documenting their identity and residence, black South Africans gathered in front of police stations without their papers. Though other protests ended peacefully, in Sharpeville, police fired into the crowd.
What is important
about Sharpeville
is not 

Please enter the model's answer to the prompt:The poet means that while the world may express its disapproval of apartheid and discrimination, it does so in a subtle and muted manner. The phrase "what the world whispers" suggests that the opposition to apartheid is not as vocal or forceful as it should be. Therefore, the correct interpretation is "The world engages in subtler forms of discrimination." The world may not openly support or endorse apartheid, but it engages in more subtle forms of discrimination or fails to take effective action against it.

The prompt is:
This question refers to the following information.
No task is more urgent than that of preserving peace. Without peace our independence means little. The rehabilitation and upbuilding of our countries will have little meaning. Our revolutions will not be allowed to run their course. What can we do? We can do much! We can inject the voice of reason into world affairs. We can mobilize all the spiritual, all the moral, all 

Please enter the model's answer to the prompt:The passage above is most associated with the formation of the non-aligned movement. The reference to mobilizing the spiritual, moral, and political strength of Asia and Africa on the side of peace aligns with the goals and principles of the non-aligned movement, which aimed to promote neutrality and independence from the Cold War superpowers and foster cooperation among developing nations. The Bandung Conference mentioned in the passage was a significant event that led to the establishment of the non-aligned movement.



# Save as a csv file

In [ ]:
prompt_answers_df = pd.DataFrame(list(zip(prompts, answers))).rename(columns={0: 'prompts', 1: 'a'})

In [ ]:
prompt_answers_df.head()  # Verify

,prompts,a
0,A fold observed in layers of sedimentary rock ...,The correct answer is (C) deposition of river ...
1,A signal from the brain to a muscle in the arm...,The signal from the brain to a muscle in the a...
2,What characteristic of DNA results in cell dif...,The characteristic of DNA that results in cell...
3,Lactose intolerance is a condition of the dige...,If more adults than children are diagnosed wit...
4,Which celestial object listed below has the gr...,The celestial object with the greatest density...


In [ ]:
model_name = "ChatGPT"
prompt_answers_df.to_csv(f'prompt_answers_{model_name}.csv', index=False)